# F-тест: Структурные изменения

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col # вывод подгонки
from scipy.stats import f # F-распределение
import seaborn as sns

## Sleep equation
Рассмотрим регрессию **sleep/60 на totwrk, age, age^2, south, smsa, marr**

* для М $sleep/60=\beta_0+\beta_1totwrk+\beta_2age+\beta_3age^2+\beta_4south+\beta_5smsa+\beta_6marr+u$
* для Ж $sleep/60=\gamma_0+\gamma_1totwrk+\gamma_2age+\gamma_3age^2+\gamma_4south+\gamma_5smsa+\gamma_6marr+v$

Будем тестировать $H_0:\beta_j=\gamma_j$ (всего $k+1$ ограничение)

Уровень значимости $\alpha=0.01$

Для тестовой статистики подговим модель
* только по М
* только по Ж
* по полному датасету

Результаты подгонки

In [ ]:
sleep_df = pd.read_csv('https://raw.githubusercontent.com/artamonoff/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
specification = 'I(sleep/60)~totwrk+age+I(age**2)+south+smsa+marr'
# подгонка по полному датасету
mod_overall = smf.ols(formula=specification, data=sleep_df).fit()
# подгонка только по М
mod_men_only = smf.ols(formula=specification, data=sleep_df[ sleep_df['male']==1 ]).fit()
# подгонка только по Ж
mod_women_only = smf.ols(formula=specification, data=sleep_df[ sleep_df['male']==0 ]).fit()
# Вывод трёх регрессий в одной таблице
summary_col(results=[mod_men_only, mod_women_only, mod_overall], stars=True, model_names=['муж', 'жен', 'Общая'],
           info_dict={'N': lambda x: x.nobs, 'F-stat': lambda x: x.fvalue})

In [ ]:
# Ингредиенты тестовой статистики
print('RSS (overall)=', mod_overall.ssr)
print('RSS (men_only)=', mod_men_only.ssr)
print('RSS (women_only)=', mod_women_only.ssr)
print('n=', mod_overall.nobs)
print('k=', mod_overall.df_model)

Тестовая статистика $$F=\frac{RSS_{overall}-RSS_{men}-RSS_{women}}{RSS_{men}+RSS_{women}}*\frac{n-2(k+1)}{k+1}$$
Критическое значение $$F_{cr}=F_{df1=k+1, df2=n-2(k+1)}(\alpha)$$

In [ ]:
F = (mod_overall.ssr-mod_men_only.ssr-mod_women_only.ssr)/(mod_men_only.ssr+mod_women_only.ssr)*(mod_overall.nobs-2*(mod_overall.df_model+1))/(mod_overall.df_model+1)
F_cr = f.ppf(q=1-0.01, dfn=mod_overall.df_model+1, dfd=mod_overall.nobs-2*(mod_overall.df_model+1))
F, F_cr

Некоторые графики

In [ ]:
sns.lmplot(data=sleep_df, x='totwrk', y='sleep', hue='male', ci=False, markers='.')

In [ ]:
sns.lmplot(data=sleep_df, x='age', y='sleep', hue='male', ci=False, markers='.', order=2)